In [2]:
import os
import time
import numpy as np
import pandas as pd
import pytz
from docker.src.measurement_stat import MEASUREMENT_SOURCE_VALUE_STATS
from datetime import datetime, timedelta, time as datetime_time, timezone
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import MinMaxScaler


VALUE_MAP = ['HR','RR','SpO2','Pulse','Temp','ABPm','ABPd','ABPs','NBPm','NBPs','NBPd','SPO2-%','SPO2-R',
'Resp','PVC','ST-II','etCO2','SpO2 r','imCO2','ST-V1','ST-I','ST-III','ST-aVF','ST-aVL','ST-aVR',
'awRR','CVPm','AoM','ST-V2','ST-V3','ST-V4','ST-V5','ST-V6','SpO2T','T1','TV','Cdyn','PEEP','RRaw',
'TVin','inO2','AoD','AoS','InsTi','MINVOL','MnAwP','PIP','MVin','PB','Poccl','Pplat',
'MV','Patm','Ppeak','Rinsp','ST-V','sInsTi','sPEEP','sTV','sTrig','sPSV','Rexp','highP',
'sAPkFl','sAWRR','sFIO2','sPIF','sMV','sO2','sRisTi','ARTd','ARTm','ARTs','PAPm','sSIMV']

MEASUREMENT_NORMALIZATION = ['mean', 'predefined']


In [3]:
data_path='./data/train'
common_path='./data/volume'

task_path='./data/volume/local_test'
# os.mkdir(task_path)
is_train=True

group_hour=1
timestep_per_data=128

measurement_normalize='mean'

condition_min_limit=0
condition_group=False

valid_size=0.2
data_split_random_seed=1235
pytest=False

In [4]:
  def extract_outcome_cohort():
    start_time = time.time()
    cohort_df = pd.read_csv(os.path.join(data_path, 'OUTCOME_COHORT.csv'), encoding='windows-1252')

    cohort_df.COHORT_START_DATE = pd.to_datetime(cohort_df.COHORT_START_DATE)
    cohort_df.COHORT_END_DATE = pd.to_datetime(cohort_df.COHORT_END_DATE)
    print("data_loader extract_outcome_cohort time:", time.time() - start_time)
    return cohort_df

  def extract_person():
    start_time = time.time()
    person_df = pd.read_csv(os.path.join(data_path, 'PERSON_NICU.csv'), encoding='windows-1252')
    person_df = pd.concat([
        person_df[['PERSON_ID', 'BIRTH_DATETIME']],
        pd.get_dummies(person_df.GENDER_SOURCE_VALUE, prefix='gender')
    ], axis=1)

    # 생일 컬럼 타입 설정
    person_df.BIRTH_DATETIME = pd.to_datetime(person_df.BIRTH_DATETIME, utc=True)
    # 여성/남성 컬럼 1개로 처리
    person_df.rename(columns={'gender_M': 'GENDER'}, inplace=True)
    if 'gender_F' in person_df.columns:
      del person_df['gender_F']

    print("data_loader extract_person time:", time.time() - start_time)
    return person_df

  def extract_condition():
    start_time = time.time()
    condition_df = pd.read_csv(os.path.join(data_path, 'CONDITION_OCCURRENCE_NICU.csv'), encoding='windows-1252',
                               usecols=['PERSON_ID', 'CONDITION_SOURCE_VALUE', 'CONDITION_START_DATETIME'])
    # Null 이거나 값이 빈 것을 날림
    condition_df = condition_df[pd.notnull(condition_df.CONDITION_SOURCE_VALUE)]
    condition_df = condition_df[condition_df.CONDITION_SOURCE_VALUE.str.len() > 0]

    if condition_group:
      condition_df.CONDITION_SOURCE_VALUE = condition_df.CONDITION_SOURCE_VALUE.str.slice(stop=3)

    # 컬럼 타입 설정
    condition_df.CONDITION_START_DATETIME = pd.to_datetime(condition_df.CONDITION_START_DATETIME, utc=True)

    print("data_loader extract_condition time:", time.time() - start_time)
    return condition_df

  def extract_measurement():
    start_time = time.time()
    measurement_df = pd.read_csv(os.path.join(data_path, 'MEASUREMENT_NICU.csv'), 
                                 encoding='windows-1252',
                                 usecols=['PERSON_ID', 'MEASUREMENT_DATETIME',
                                          'MEASUREMENT_SOURCE_VALUE', 'VALUE_AS_NUMBER']
                                 )
#     if measurement_normalize == MEASUREMENT_NORMALIZATION[0]:
#       # source_value 맵핑
#       source_value_invert_map = {}
#       for new_value in MEASUREMENT_SOURCE_VALUE_MAP:
#         for table_value in MEASUREMENT_SOURCE_VALUE_MAP[new_value]:
#           source_value_invert_map[table_value] = new_value
#       measurement_df.MEASUREMENT_SOURCE_VALUE = measurement_df.MEASUREMENT_SOURCE_VALUE.replace(source_value_invert_map)

      # 맵핑이된 정보만 남긴다
    measurement_df = measurement_df[measurement_df.MEASUREMENT_SOURCE_VALUE.isin(VALUE_MAP)]

    # 컬럼 타입 설정
    measurement_df.MEASUREMENT_DATETIME = pd.to_datetime(measurement_df.MEASUREMENT_DATETIME, utc=True)

    # source_value별 평균값 추출
    if is_train:
      measurement_mean_df = measurement_df.groupby('MEASUREMENT_SOURCE_VALUE').VALUE_AS_NUMBER.mean()
      measurement_mean_df.to_pickle(os.path.join(common_path, 'measurement_mean.pkl'))
    else:
      # inference일 경우 저장된 걸 불러온다
      measurement_mean_df = pd.read_pickle(os.path.join(common_path, 'measurement_mean.pkl'))

    print("data_loader extract_measurement time:", time.time() - start_time)
    return measurement_df, measurement_mean_df
cohort_df = extract_outcome_cohort()
person_df = extract_person()
condition_df = extract_condition()
measurement_df,measurement_mean_df = extract_measurement()

data_loader extract_outcome_cohort time: 0.011013984680175781
data_loader extract_person time: 0.0070133209228515625
data_loader extract_condition time: 0.006035566329956055
data_loader extract_measurement time: 2.281010866165161


In [5]:
  def groupby_hour_condition( condition_df):
    start_time = time.time()

    condition_df['CONDITION_DATE'] = condition_df.CONDITION_START_DATETIME.dt.date
    condition_df['CONDITION_DATE'] = pd.to_datetime(condition_df.CONDITION_DATE, utc=True)

    if is_train and condition_min_limit > 0:
      condition_group = condition_df.groupby('CONDITION_SOURCE_VALUE').PERSON_ID.count()
      condition_group = condition_group[condition_group > condition_min_limit].index

      condition_df = condition_df[condition_df.CONDITION_SOURCE_VALUE.isin(condition_group)]

    # 진단은 시간이 없다. 당일의 마지막에 진단 받은걸로 가정한다
    condition_df['HOURGRP'] = 23 // group_hour

    group_cols = ['PERSON_ID', 'CONDITION_DATE', 'HOURGRP', 'CONDITION_SOURCE_VALUE']

    condition_df['DUMMY'] = condition_df['CONDITION_SOURCE_VALUE']
    condition_df = condition_df.groupby(group_cols) \
        .DUMMY.count().unstack().reset_index().fillna(0)

    condition_df = condition_df.rename(columns={'CONDITION_DATE': 'DATE'})

    condition_col_filename = os.path.join(task_path, 'condition_cols.npy')
    if is_train:
      # 컬럼 이름 저장
      np.save(condition_col_filename, np.array(condition_df.columns))
    else:
      # 컬럼 로드
      condition_cols = np.load(condition_col_filename, allow_pickle=True)
      new_condition_list = []
      for col in condition_cols:
        if col in condition_df.columns:
          new_condition_list.append(condition_df[col])
        else:
          new_condition_list.append(pd.Series([0] * condition_df.shape[0]))

      condition_df = pd.concat(new_condition_list, axis=1)
      condition_df.columns = condition_cols
    print("data_loader groupby_hour_condition time:", time.time() - start_time)
    return condition_df

In [6]:
condition_df = groupby_hour_condition(condition_df)

data_loader groupby_hour_condition time: 0.01300191879272461


In [7]:
  def _clip_measurement(measurement_source_value, value_as_number):
    if value_as_number > MEASUREMENT_SOURCE_VALUE_STATS[measurement_source_value]['95%']:
      value_as_number = MEASUREMENT_SOURCE_VALUE_STATS[measurement_source_value]['95%']
    elif value_as_number < MEASUREMENT_SOURCE_VALUE_STATS[measurement_source_value]['5%']:
      value_as_number = MEASUREMENT_SOURCE_VALUE_STATS[measurement_source_value]['5%']
    return value_as_number

  def groupby_hour_measurement(measurement_df):
    start_time = time.time()
    # timestamp로 join 하기 위하여 시간 포맷을 utc로 통일
    measurement_df['MEASUREMENT_DATE'] = measurement_df.MEASUREMENT_DATETIME.dt.date
    measurement_df['MEASUREMENT_DATE'] = pd.to_datetime(measurement_df.MEASUREMENT_DATE, utc=True)

    measurement_df['MEASUREMENT_HOUR'] = measurement_df.MEASUREMENT_DATETIME.dt.hour
    measurement_df['MEASUREMENT_HOURGRP'] = measurement_df.MEASUREMENT_HOUR // group_hour

    # 평균값 이용하여 Normalize
    if measurement_normalize == MEASUREMENT_NORMALIZATION[0]:
      measurement_df = pd.merge(measurement_df,
                                measurement_mean_df.reset_index().rename(
                                    columns={'VALUE_AS_NUMBER': 'MEAN_VALUE'}),
                                on='MEASUREMENT_SOURCE_VALUE', how='left')
      measurement_df.VALUE_AS_NUMBER = measurement_df.VALUE_AS_NUMBER / measurement_df.MEAN_VALUE
    # 생체신호 범위를 이용하여 Normalize
    elif measurement_normalize == MEASUREMENT_NORMALIZATION[1]:
      measurement_df.VALUE_AS_NUMBER = measurement_df.apply(lambda row:
                                                            _clip_measurement(
                                                                row['MEASUREMENT_SOURCE_VALUE'],
                                                                row['VALUE_AS_NUMBER']),
                                                            axis=1)

      # TODO
    group_cols = ['PERSON_ID', 'MEASUREMENT_DATE', 'MEASUREMENT_HOURGRP', 'MEASUREMENT_SOURCE_VALUE']
    agg_list = ['count', 'min', 'max', 'mean', 'std', 'var']
    measurement_df['VALUE_DIFF'] = measurement_df.groupby(group_cols).VALUE_AS_NUMBER.diff()

    measurement_diff_df = pd.pivot_table(measurement_df, 
                                         values='VALUE_DIFF', index=group_cols[:-1],
                                         columns='MEASUREMENT_SOURCE_VALUE', aggfunc=['mean','max','min'])

    measurement_diff_df.columns = [('diff', '{}_{}'.format(v[0],v[1])) for v in measurement_diff_df.columns]

    measurement_df = measurement_df.groupby(group_cols).VALUE_AS_NUMBER.agg(agg_list).fillna(0).unstack().fillna(method='ffill').fillna(method='bfill')

    measurement_df = pd.concat([measurement_df, measurement_diff_df], axis=1).reset_index()

    if measurement_df.isnull().sum().sum() >0:
        print("there is Na after interpolation")
        measurement_df = measurement_df.fillna(0)
        
    # 사용한 후 삭제
    del measurement_diff_df
    # 컬럼 이름 정제 (그룹화 하기 쉽게)
    new_cols = []
    for col in measurement_df.columns:
      
      if col[1] == '':
        new_cols.append(col[0])
      elif col[0] in agg_list + ['diff']:
        new_cols.append((col[1], col[0]))
    measurement_df.columns = new_cols

#     #minmax scale
#     scaler = MinMaxScaler(feature_range=(-1,1))
#     scaler = scaler.fit(measurement_df.iloc[:,3:])
#     measurement_df.iloc[:,3:] = scaler.transform(measurement_df.iloc[:,3:])
    
    measurement_df = measurement_df.rename(columns={'MEASUREMENT_DATE': 'DATE',
                                                    'MEASUREMENT_HOURGRP': 'HOURGRP'})

    measurement_col_filename = os.path.join(task_path, 'measurement_cols.npy')
    if is_train:
      # 컬럼 이름 저장
      np.save(measurement_col_filename, np.array(measurement_df.columns))
    else:
      # 컬럼 로드
      measurement_cols = np.load(measurement_col_filename, allow_pickle=True)
      new_measurement_list = []
      for col in measurement_cols:
        if col in measurement_df.columns:
          new_measurement_list.append(measurement_df[col])
        else:
          new_measurement_list.append(pd.Series([0] * measurement_df.shape[0]))

      measurement_df = pd.concat(new_measurement_list, axis=1)
      measurement_df.columns = measurement_cols
    print("data_loader groupby_hour_measurement time:", time.time() - start_time)
    return measurement_df

In [8]:
measurement_df = groupby_hour_measurement(measurement_df)

there is Na after interpolation
data_loader groupby_hour_measurement time: 7.197989463806152


### Autoencoder 작업중 

In [1]:
from docker.src.data_loader import DataLoader
import os
task_path='./data/volume/local_test'
data_loader = DataLoader(data_path=os.path.join('./data', 'train'),
                         common_path=os.path.join("./data", 'volume'),
                         measurement_normalize='mean',
                         is_train = True,
                         task_path=task_path,
                        switch = False)

C:\Users\podkd\.conda\envs\park\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\podkd\.conda\envs\park\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\podkd\.conda\envs\park\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\podkd\.conda\envs\park\lib\site-packages\tensorflow\python\framework\dtypes.

data_loader extract_outcome_cohort time: 0.009999513626098633
data_loader extract_person time: 0.006041765213012695
data_loader extract_condition time: 0.00601649284362793
data_loader extract_measurement time: 2.2839856147766113
data_loader groupby_hour_condition time: 0.0169980525970459
there is Na after interpolation
data_loader groupby_hour_measurement time: 6.957036018371582
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 108)               0         
_________________________________________________________________
encoder1 (Dense)             (None, 128)               13952     
_________________________________________________________________
encoder2 (Dense)             (None, 64)                8256      
_________________________________________________________________
decoder1 (Dense)             (None, 108)               7020      
Total p

In [23]:
from docker.src.model import Autoencoder
from keras.callbacks import EarlyStopping, ModelCheckpoint

train_measure, valid_measure = train_test_split(measurement_df,
                                                      train_size=(1 - valid_size),
                                                      test_size=valid_size,
                                                      random_state=data_split_random_seed)

autoen = Autoencoder(train_measure.iloc[:,3:])

callbacks = [
ModelCheckpoint(filepath=os.path.join(task_path, 'encoder-{epoch:02d}-{val_loss:2f}.hdf5'),
            monitor='val_loss',
            mode='min',
            save_best_only=True,
            save_weights_only=False,
            verbose=True
),
EarlyStopping(monitor='val_loss', min_delta=0, patience=20, verbose=2, mode='auto')
]

autoen.train(train_measure.iloc[:,3:],
             valid_measure.iloc[:,3:], 
             epochs = 10, 
             batch_size = int(np.floor(len(train_measure.iloc[:,3:]))),
             verbose = 2,
            callbacks = callbacks)

Model: "model_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        (None, 108)               0         
_________________________________________________________________
encoder1 (Dense)             (None, 128)               13952     
_________________________________________________________________
encoder2 (Dense)             (None, 64)                8256      
_________________________________________________________________
decoder1 (Dense)             (None, 108)               7020      
Total params: 29,228
Trainable params: 29,228
Non-trainable params: 0
_________________________________________________________________
None
Train on 1824 samples, validate on 457 samples
Epoch 1/10
 - 0s - loss: 726.2963 - val_loss: 717.0902

Epoch 00001: val_loss improved from inf to 717.09021, saving model to ./data/volume/local_test\encoder-01-717.090210.hdf5
Epoch 2/10
 - 0s - los

In [25]:
output=pd.DataFrame(autoen.predict(measurement_df.iloc[:,3:]))

In [26]:
measurement_as = pd.concat([measurement_df.iloc[:,:3], output],axis=1)

In [27]:
measurement_as

,PERSON_ID,DATE,HOURGRP,0,1,2,3,4,5,6,...,54,55,56,57,58,59,60,61,62,63
0,1.000000e+01,2036-04-22 00:00:00+00:00,23,0.0,1.463015,3.018350,3.404919,0.0,0.000000,0.175673,...,0.0,0.0,4.442196,0.0,0.0,0.000000,0.768957,0.0,0.561999,2.057318
1,1.000000e+01,2036-04-23 00:00:00+00:00,0,0.0,1.724736,2.757065,2.549405,0.0,0.000000,0.289711,...,0.0,0.0,4.725230,0.0,0.0,0.000000,0.630201,0.0,0.966726,2.557830
2,1.000000e+01,2036-04-23 00:00:00+00:00,1,0.0,1.764820,2.749552,2.584313,0.0,0.000000,0.364534,...,0.0,0.0,4.730961,0.0,0.0,0.000000,0.641720,0.0,0.922184,2.560203
3,1.000000e+01,2036-04-23 00:00:00+00:00,2,0.0,1.772781,2.736174,2.570029,0.0,0.000000,0.371790,...,0.0,0.0,4.730395,0.0,0.0,0.000000,0.642221,0.0,0.916005,2.567777
4,1.000000e+01,2036-04-23 00:00:00+00:00,3,0.0,1.793007,2.718867,2.597193,0.0,0.000000,0.411221,...,0.0,0.0,4.739558,0.0,0.0,0.000000,0.653624,0.0,0.879824,2.573025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2276,4.771510e+16,2036-05-05 00:00:00+00:00,22,0.0,1.675696,3.037452,2.716092,0.0,0.000000,0.014164,...,0.0,0.0,4.406900,0.0,0.0,0.000000,0.239990,0.0,1.138141,2.400742
2277,4.771510e+16,2036-05-05 00:00:00+00:00,23,0.0,1.527802,3.248977,2.450779,0.0,0.000000,0.000000,...,0.0,0.0,4.406554,0.0,0.0,0.000000,0.237040,0.0,1.379658,2.302514
2278,4.771510e+16,2036-05-06 00:00:00+00:00,0,0.0,1.784357,3.198696,2.215753,0.0,0.000000,0.413760,...,0.0,0.0,3.351112,0.0,0.0,0.000000,0.055041,0.0,1.422162,2.291072
2279,4.771510e+16,2036-05-06 00:00:00+00:00,1,0.0,2.017043,3.245299,2.080878,0.0,0.000000,0.500519,...,0.0,0.0,3.467436,0.0,0.0,0.000000,0.255916,0.0,1.631893,2.546199


In [32]:
autoen.model.load_weights(model_path)

In [34]:
autoen.model.layers

In [25]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from keras.models import model_from_json 


model_path = tf.train.latest_checkpoint(task_path)
if model_path is None:
  file_name = sorted([file_name for file_name in os.listdir(task_path) if file_name.endswith('.hdf5') and file_name.startswith('encoder')])[-1]
  model_path = os.path.join(task_path, file_name)

model = model.load_weights(model_path)


NameError: name 'load_weights' is not defined

In [23]:
file_name

'encoder-10-725.665955.hdf5'

In [37]:
from keras.layers import Input
from keras.models import Model

train_measure, valid_measure = train_test_split(measurement_df,
                                                      train_size=(1 - valid_size),
                                                      test_size=valid_size,
                                                      random_state=data_split_random_seed)

input_img = Input(shape=(train_measure.iloc[:,3:].shape[1],))
layer1=autoen.model.layers[1]
layer2=autoen.model.layers[2]

encoder= Model(input_img, layer2(layer1(input_img)))
output=encoder.predict(train_measure.iloc[:,3:])


In [38]:
output

array([[0.        , 0.        , 1.9231129 , ..., 2.1713006 , 0.4146613 ,
        2.4669633 ],
       [0.        , 0.        , 1.7895968 , ..., 2.17478   , 0.41060454,
        2.4875143 ],
       [0.        , 0.        , 1.824985  , ..., 2.1258125 , 0.11797369,
        2.0025125 ],
       ...,
       [0.        , 0.        , 1.8242193 , ..., 2.1266065 , 0.11741423,
        2.0028756 ],
       [0.        , 0.        , 1.8244132 , ..., 2.126528  , 0.11736455,
        2.0026877 ],
       [0.        , 0.        , 1.8885305 , ..., 2.1614344 , 0.43662977,
        2.492919  ]], dtype=float32)

In [ ]:
    input_img = Input(shape=(self.train_measure.shape[1],))
    layer1=self.model.layers[1]
    layer2=self.model.layers[2]

    encoder= Model(input_img, layer2(layer1(input_img)))
    output=encoder.predict(total_measure)
    return output

In [26]:
def autoencoder():
    train_measure, valid_measure = train_test_split(measurement_df,
                                                          train_size=(1 - valid_size),
                                                          test_size=valid_size,
                                                          random_state=data_split_random_seed)
    autoen = Autoencoder(train_measure.iloc[:,3:])
    autoen.train(train_measure.iloc[:,3:],
             valid_measure.iloc[:,3:], 
             epochs = 10, 
             batch_size = int(np.floor(len(train_measure.iloc[:,3:]))),
             verbose = 2,
            callbacks = [])
    self.model = autoen
    output = autoen.predict(measurement_df.iloc[:,3:])
    mesurement_df[:,3:] = output
    return measurement_df

In [ ]:
autoen.train(train_measure.iloc[:,3:],
             valid_measure.iloc[:,3:], 
             epochs = 10, 
             batch_size = int(np.floor(len(train_measure.iloc[:,3:]))),
             verbose = 2,
            callbacks = [])

output = autoen.predict(measurement_df.iloc[:,3:])

#preprocessing

In [ ]:
  def make_person_sequence():
    start_time = time.time()
    # 환자별로 데이터의 시작시간과 종료시간을 구한다.
    timerange_df = cohort_df.groupby('SUBJECT_ID').agg({'COHORT_START_DATE': 'min', 'COHORT_END_DATE': 'max'})
    timerange_df['START_DATE'] = timerange_df.COHORT_START_DATE.dt.date
    timerange_df['START_HOURGRP'] = timerange_df.COHORT_START_DATE.dt.hour // group_hour
    timerange_df['END_DATE'] = timerange_df.COHORT_END_DATE.dt.date
    timerange_df['END_HOURGRP'] = timerange_df.COHORT_END_DATE.dt.hour // group_hour
    timerange_df = timerange_df.drop(['COHORT_START_DATE', 'COHORT_END_DATE'], axis=1)

    demographic_ary = person_df.sort_values('PERSON_ID', ascending=True).values
    condition_ary = condition_df.sort_values(['PERSON_ID', 'DATE', 'HOURGRP'], ascending=True).values
    measurement_ary = measurement_df.sort_values(['PERSON_ID', 'DATE', 'HOURGRP'], ascending=True).values
    timerange_ary = timerange_df.sort_values('SUBJECT_ID', ascending=True).reset_index().values

    demographic_cols = ["AGE_HOUR", "GENDER"]
    condition_cols = condition_df.columns[3:]
    measurement_cols = measurement_df.columns[3:]

    # 빈 Time Range 없게 시간대 정보를 채움
    max_hourgrp = (24 // group_hour) - 1

    key_list = []
    for person_id, start_date, start_hourgrp, end_date, end_hourgrp in timerange_ary:
      cur_date = start_date
      cur_hourgrp = start_hourgrp

      while True:
        key_list.append((person_id, cur_date, cur_hourgrp))

        cur_hourgrp += 1                  # 1 그룹시간만큼 탐색
        if cur_hourgrp > max_hourgrp:     # 다음 날짜로 넘어감
          cur_date = cur_date + timedelta(days=1)
          cur_hourgrp = 0

        if cur_date > end_date or \
           (cur_date == end_date and cur_hourgrp >= end_hourgrp):
          # 끝까지 탐색함
          break

    # 시간대 정보에 따라 데이터를 채워 넣는다
    demographic_idx = condition_idx = measurement_idx = 0
    prev_person_id = None
    prev_conditions = None

    data_cols = list(demographic_cols) + list(measurement_cols) + list(condition_cols)
    data_list = np.zeros((len(key_list), len(data_cols)), dtype=np.float32)
    for idx, row in enumerate(key_list):
      person_id, date, hourgrp = row

      col_start_idx = col_end_idx = 0
      col_end_idx += len(demographic_cols)
      # Demographic 추가
      while True:
        if demographic_idx >= len(demographic_ary):
          break

        demographic_row = demographic_ary[demographic_idx]
        demographic_person_id = demographic_row[0]
        # 시간 계산을 위해 tz를 동일하게 맞춤.
        demographic_age = datetime.combine(date, datetime_time(hour=hourgrp, tzinfo=timezone.utc)).astimezone(
            pytz.utc) - demographic_row[1]
        demographic_gender = demographic_row[2]
        demographic_data = [demographic_age.total_seconds() // 3600., demographic_gender]

        state = 0       # 0: 다음 데이터 탐색 1: 맞는 데이터 찾음 2: 맞는 데이터 없음
        if demographic_person_id > person_id:       # 다음 환자로 넘어감
          state = 2
        elif demographic_person_id == person_id:  # 맞는 데이터
          state = 1

        if state == 0:                  # 계속 탐색
          demographic_idx += 1
        elif state == 1:                # 데이터 찾음
          data_list[idx, col_start_idx:col_end_idx] = demographic_data
          break
        elif state == 2:                # 맞는 데이터가 없음
          break

      # Measurement 탐색
      col_start_idx = col_end_idx
      col_end_idx += len(measurement_cols)
      while True:
        if measurement_idx >= len(measurement_ary):
          break

        measurement_row = measurement_ary[measurement_idx]
        measurement_person_id = measurement_row[0]
        measurement_date = measurement_row[1]
        measurement_hourgrp = measurement_row[2]
        measurement_data = measurement_row[3:]

        state = 0       # 0: 다음 데이터 탐색 1: 맞는 데이터 찾음 2: 맞는 데이터 없음
        if measurement_person_id > person_id:       # 다음 환자로 넘어감
          state = 2
        elif measurement_person_id == person_id:
          if measurement_date.date() > date:               # 다음 날짜로 넘어감
            state = 2
          elif measurement_date.date() == date:
            if measurement_hourgrp > hourgrp:       # 다음 그룹시간으로 넘어감
              state = 2
            elif measurement_hourgrp == hourgrp:    # 맞는 데이터
              state = 1

        if state == 0:                  # 계속 탐색
          measurement_idx += 1
        elif state == 1:                # 데이터 찾음
          data_list[idx, col_start_idx:col_end_idx] = measurement_data
          measurement_idx += 1
          break
        elif state == 2:                # 맞는 데이터가 없음
          break

      # Condition 탐색
      col_start_idx = col_end_idx
      col_end_idx += len(condition_cols)
      # 이전과 다른 환자임. condition정보 reset
      if prev_person_id != person_id:
        prev_conditions = np.array([0] * len(condition_cols))

      while True:
        if condition_idx >= len(condition_ary):
          break

        condition_row = condition_ary[condition_idx]
        condition_person_id = condition_row[0]
        condition_date = condition_row[1]
        condition_hourgrp = condition_row[2]
        condition_data = condition_row[3:]

        state = 0       # 0: 다음 데이터 탐색 1: 맞는 데이터 찾음 2: 맞는 데이터 없음
        if condition_person_id > person_id:       # 다음 환자로 넘어감
          state = 2
        elif condition_person_id == person_id:
          if condition_date.date() > date:               # 다음 날짜로 넘어감
            state = 2
          elif condition_date.date() == date:
            if condition_hourgrp > hourgrp:       # 다음 그룹시간으로 넘어감
              state = 2
            elif condition_hourgrp == hourgrp:    # 맞는 데이터
              state = 1

        if state == 0:                  # 계속 탐색
          condition_idx += 1
        elif state == 1:                # 데이터 찾음
          # 이전 Condition 정보와 나중 Condition 정보를 합친다
          prev_conditions = np.array(prev_conditions) + np.array(condition_data)
          data_list[idx, col_start_idx:col_end_idx] = prev_conditions
          condition_idx += 1
          break
        elif state == 2:                # 맞는 데이터가 없음
          break

      prev_person_id = person_id

    feature_ary = data_list
    feature_key_df = pd.DataFrame(key_list, columns=['PERSON_ID', 'DATE', 'HOURGRP'])
    print("data_loader make_person_sequence time:", time.time() - start_time)
    return feature_ary, feature_key_df

In [ ]:
feature_ary, feature_key_df = make_person_sequence()

In [ ]:

def make_data():
    start_time = time.time()
    # 빠른 서치를 위하여 데이터 정렬
    # 가장 마지막 시점이 먼저 오도록 반대로 정렬
    global cohort_df, feature_key_df, feature_ary
    cohort_df = cohort_df.sort_values(['SUBJECT_ID', 'COHORT_END_DATE'], ascending=[True, False])
    feature_key_df = feature_key_df.sort_values(['PERSON_ID', 'DATE', 'HOURGRP'], ascending=[True, False, False])
    feature_ary = feature_ary[feature_key_df.index]
    feature_key_ary = feature_key_df.values
    
    cols = ['SUBJECT_ID', 'COHORT_END_DATE']
    if is_train:
      cols.append('LABEL')

    x_list = []
    y_list = []
    key_list = []
    feature_idx = 0

    for row in cohort_df[cols].values:
      subject_id = row[0]
      cohort_end_date = row[1]

      # key에 맞는 data feature 찾는다
      while True:
        person_id, feature_date, feature_hourgrp = feature_key_ary[feature_idx]
        feature_row = feature_ary[feature_idx]

        feature_datetime = datetime(feature_date.year,
                                    feature_date.month,
                                    feature_date.day,
                                    feature_hourgrp * group_hour)
        if person_id == subject_id and feature_datetime < cohort_end_date:
          # 같은 환자이고 cohort_end_date보다 먼저 발생한 데이터이면
          # 맞는 데이터
          each_x_list = []
          for timestep in range(timestep_per_data):
            if feature_idx + timestep >= len(feature_ary):
              break
            timestep_person_id = feature_key_ary[feature_idx + timestep][0]
            timestep_row = feature_ary[feature_idx + timestep]
            if timestep_person_id == subject_id:
              timestep_data = timestep_row
              each_x_list.append(timestep_data)
            else:
              break
          # 가장 나중 데이터부터 each_x_list에 넣었으니 데이터에 넣을땐 반대로
          x_list.append(np.array(each_x_list)[::-1])
          break
        elif person_id > subject_id:
          # 데이터를 못찾음. 다음 환자로 넘어가버렸다
          print("Person's data not found", subject_id)
          feature_data = feature_row
          x_list.append(np.array([[0] * len(feature_data)]))
          break
        else:
          # 탐색이 더 필요함
          feature_idx += 1

      # y 추가
      if is_train:
        label = row[2]
        y_list.append(label)

      key_list.append((row[0], row[1]))

    x = np.array(x_list)
    y = np.array(y_list) if is_train else None
    key = pd.DataFrame(key_list, columns=['SUBJECT_ID', 'COHORT_END_DATE'])
    print("X", x.shape)
    if y is not None:
      print("Y", y.shape)
    print("Key", key.shape)
    print("data_loader make_data time:", time.time() - start_time)
    return x,y,key

In [ ]:
x,y,key = make_data()

In [231]:
  def _stratified_shuffle():
    whole_patient = set(key.SUBJECT_ID.unique())
    true_patient = set(key.loc[np.where(y == 1)[0], ].SUBJECT_ID.unique())
    false_patient = whole_patient - true_patient

    true_train_patient, true_valid_patient = train_test_split(list(true_patient),
                                                              train_size=(1 - valid_size),
                                                              test_size=valid_size,
                                                              random_state=data_split_random_seed)

    false_train_patient, false_valid_patient = train_test_split(list(false_patient),
                                                                train_size=(1 - valid_size),
                                                                test_size=valid_size,
                                                                random_state=data_split_random_seed)

    true_train_x = x[key.SUBJECT_ID.isin(true_train_patient)]
    true_valid_x = x[key.SUBJECT_ID.isin(true_valid_patient)]
    true_train_y = y[key.SUBJECT_ID.isin(true_train_patient)]
    true_valid_y = y[key.SUBJECT_ID.isin(true_valid_patient)]
    
    true_x = true_train_x[np.where(true_train_y==1)]
    
    true_train_x = x[key.SUBJECT_ID.isin(true_train_patient)]
    true_train_x = x[key.SUBJECT_ID.isin(true_train_patient)]
    true_train_x = x[key.SUBJECT_ID.isin(true_train_patient)]
    
    train_patient = np.concatenate([true_train_patient, false_train_patient])
    valid_patient = np.concatenate([true_valid_patient, false_valid_patient])

    train_x = x[key.SUBJECT_ID.isin(train_patient)]
    train_y = y[key.SUBJECT_ID.isin(train_patient)]

    valid_x = x[key.SUBJECT_ID.isin(valid_patient)]
    valid_y = y[key.SUBJECT_ID.isin(valid_patient)]
    return train_x, train_y, valid_x, valid_y

In [1]:
np.where(y==1)[0]

NameError: name 'np' is not defined

In [268]:
model_input = Input((None, x3.shape[2]))
x = model_input

x = Masking(mask_value=0.0)(x)

x = Dense(16, activation = 'tanh')(x)
rnn_layers = [32]
for idx, node in enumerate(rnn_layers):
  return_sequences = False if idx == len(rnn_layers) - 1 else True

  x = GRU(node, activation = 'relu',
          return_sequences=return_sequences)(x)
  x = BatchNormalization()(x)
  x = Dropout(0.5)(x)

x = Dense(1, kernel_initializer=TruncatedNormal(stddev=0.01))(x)

model_output = Activation('sigmoid')(x)
loss = 'binary_crossentropy'

optimizer = Adam(learning_rate=0.001)

model = Model(model_input, model_output)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])


In [263]:
x3  =pad_sequences(x)

In [272]:
y.shape

(592,)

In [273]:
x3.shape[0]

592

In [256]:
import os
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GRU, Input, Masking, Dropout, Dense, Activation, BatchNormalization
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model

In [ ]:
Xt_time = []
Xt_static = []
yt_time = []

## positive data gathering
for i in range(len(list_time_X)):
    Xt_time_i = list_time_X[i]
    yt_time_i = list_time_y[i]
    Xt, yt = time_series_generator(Xt_time_i,yt_time_i)
    time_static_data = list_time_static[i]

    for n in range(len(Xt)):
        Xt_time.append(Xt[n])
        Xt_static.append(time_static_data)
        yt_time.append(yt[n])

positive_index = [i for i,result in enumerate(yt_time) if result==1]
negative_index = [i for i,result in enumerate(yt_time) if result==0]

positive_x = [Xt_time[i] for i in positive_index]
positive_x_static = [Xt_static[i] for i in positive_index]
positive_y = [yt_time[i] for i in positive_index]

negative_x = [Xt_time[i] for i in negative_index]
negative_x_static = [Xt_static[i] for i in negative_index]
negative_y = [yt_time[i] for i in negative_index]

true_X_time = np.array(positive_x, dtype="float32").reshape(-1,window,feature)
true_X_static = np.array(positive_x_static, dtype="float32").reshape(-1,38)
true_y_total = np.array(positive_y, dtype="float32").reshape(-1,)
false_X_time = np.array(negative_x, dtype="float32").reshape(-1,window,feature)
false_X_static = np.array(negative_x_static, dtype="float32").reshape(-1,38)
false_y_total = np.array(negative_y, dtype="float32").reshape(-1,)

In [277]:
model.fit(x3, y,
                   epochs=4,
                   verbose=1,
                   batch_size=100,
                   validation_data=[x3,y],
                   callbacks=[]
                   )

Train on 592 samples, validate on 592 samples
Epoch 1/4
592/592 [==============================] - 2s 3ms/sample - loss: 0.5255 - acc: 0.9139 - val_loss: 0.5338 - val_acc: 0.9375
Epoch 2/4
592/592 [==============================] - 2s 3ms/sample - loss: 0.5190 - acc: 0.9139 - val_loss: 0.5214 - val_acc: 0.9375
Epoch 3/4
592/592 [==============================] - 2s 3ms/sample - loss: 0.5015 - acc: 0.9409 - val_loss: 0.5076 - val_acc: 0.9375
Epoch 4/4
592/592 [==============================] - 2s 3ms/sample - loss: 0.4874 - acc: 0.9426 - val_loss: 0.4929 - val_acc: 0.9375


In [269]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, None, 122)]       0         
_________________________________________________________________
masking_3 (Masking)          (None, None, 122)         0         
_________________________________________________________________
dense_3 (Dense)              (None, None, 16)          1968      
_________________________________________________________________
gru_3 (GRU)                  (None, 32)                4704      
_________________________________________________________________
batch_normalization_1 (Batch (None, 32)                128       
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 33  

In [232]:
train_x, train_y, valid_x, valid_y = _stratified_shuffle()

ValueError: With n_samples=1, test_size=0.2 and train_size=0.8, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [ ]:

random_sampling_size = 600
fraction_per_case = 0.2
true_false_ratio = 1

class file_generator():
    "Generating model"
    def __init__(self, window, time_it, feature,
                 true_X_time, true_X_static,true_y_total,
                 false_X_time, false_X_static, false_y_total,
                 list_time_Xn, list_time_yn, list_time_nstatic, 
                 random_sampling_size, fraction_per_case, true_false_ratio):
        self.window = window
        self.time_it = time_it
        self.feature = feature
        self.true_X_time = true_X_time
        self.true_X_static = true_X_static
        self.true_y_total = true_y_total
        self.false_X_time = false_X_time
        self.false_X_static = false_X_static
        self.false_y_total = false_y_total
        self.list_time_Xn = list_time_Xn
        self.list_time_yn = list_time_yn
        self.list_time_nstatic = list_time_nstatic
        self.random_sampling_size = random_sampling_size
        self.fraction_per_case = fraction_per_case
        self.true_false_ratio = true_false_ratio


    def get_data(self, index):
        Xn_time = []
        Xn_static = []
        yn_time = []

        random_sampling = np.random.choice(len(self.list_time_Xn), size= self.random_sampling_size)
        ## negative data gathering
        for i in random_sampling:
            X_time_n = self.list_time_Xn[i]
            y_time_n = self.list_time_yn[i]
            Xn, yn = self.time_series_generator(X_time_n,y_time_n)
            time_static_data = self.list_time_nstatic[i]

            for n in range(len(Xn)):
                Xn_time.append(Xn[n])
                Xn_static.append(time_static_data)
                yn_time.append(yn[n])

        np_Xn_time = np.array(Xn_time, dtype="float32").reshape(-1,self.window,self.feature)
        np_Xn_static = np.array(Xn_static, dtype="float32").reshape(-1,38)
        np_yn_time = np.array(yn_time, dtype="float32").reshape(-1,)

        batch_false_list = np.random.choice(len(self.false_X_time), size=int(np.floor(len(self.true_X_time)*self.true_false_ratio)))

        batch_false_X = self.false_X_time[batch_false_list]
        batch_false_Xs = self.false_X_static[batch_false_list]
        batch_false_y = self.false_y_total[batch_false_list]

        batch_X=np.concatenate([np_Xn_time,batch_false_X, self.true_X_time],axis=0)
        batch_static_X=np.concatenate([np_Xn_static,batch_false_Xs, self.true_X_static], axis=0)
        batch_y=np.concatenate([np_yn_time,batch_false_y, self.true_y_total],axis=0)

        batch_X, batch_static_X, batch_y = self.shuffling(batch_X, batch_static_X, batch_y)

        return batch_X, batch_static_X, batch_y


    def time_series_generator(self, x,y):
        Xn=[]
        yn=[]
        size_x = len(x)-self.time_it
        random_selection = np.random.choice(size_x, size = int(np.floor(size_x*self.fraction_per_case)))
        for n in random_selection:
            if n+1>self.window:
                X_train=x[n+1-self.window:n+1]
            else:
                X_train=x[0:n+1]
                X_train=np.pad(X_train, mode='constant', pad_width=((0,self.window-X_train.shape[0]),(0,0)),\
                               constant_values=-5)

            Xn.append(X_train)
            y_train=y[n+self.time_it]
            yn.append(y_train)

        return Xn, yn

    def shuffling(self, a, b, c):
        assert len(a) == len(b)
        p = np.random.permutation(len(a))
        return a[p], b[p], c[p]
